In [ ]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


: 

In [ ]:
df = pd.read_csv("train.csv")
df.head()


In [ ]:
df = df[['Pclass', 'Sex', 'Age', 'Fare', 'Siblings/Spouses Aboard', 'Parents/Children Aboard', 'Survived']]

In [ ]:
df.loc[:, 'Age'] = df['Age'].fillna(df['Age'].median())

# Explicitly cast 'Sex' to object (string) dtype before filling NaNs to avoid FutureWarning
df.loc[:, 'Sex'] = df['Sex'].astype(str)

sex_mode = df['Sex'].mode()
if not sex_mode.empty:
    df.loc[:, 'Sex'] = df['Sex'].fillna(sex_mode[0])
else:
    # Fallback if 'Sex' column is entirely NaN. Assuming 'male' as a reasonable default.
    df.loc[:, 'Sex'] = df['Sex'].fillna('male')
# 'Embarked' column is not in this dataset, so skipping fillna for it.

In [ ]:
df.loc[:, 'Sex'] = df['Sex'].map({'male': 0, 'female': 1}).fillna(0).astype(int)
# 'Embarked' column is not in this dataset, so skipping mapping for it.

In [ ]:
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)


In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


In [ ]:
with open("titanic_survival_model.pkl", "wb") as file:
    pickle.dump((model, scaler), file)

print("Model saved successfully")


In [ ]:
import requests

# URL for the Titanic train.csv dataset (common Kaggle source)
url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"

# Download the file
response = requests.get(url)
with open("train.csv", "wb") as f:
    f.write(response.content)

print("train.csv downloaded successfully!")

In [ ]:
with open("titanic_survival_model.pkl", "rb") as file:
    loaded_model, loaded_scaler = pickle.load(file)

# Example sample with updated features: Pclass, Sex, Age, Fare, Siblings/Spouses Aboard, Parents/Children Aboard
sample = np.array([[3, 0, 22, 7.25, 1, 0]])
sample_scaled = loaded_scaler.transform(sample)
loaded_model.predict(sample_scaled)